In [1]:
# load Article_top10_Data.csv
import pandas as pd
import numpy as np

df = pd.read_csv('Article_top10_Data.csv', index_col=False)
df.head()

,Unnamed: 0,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,ReceiptRowNumber,Time,ArticleNumber,ArticleName,Quantity,Timestamp,Month,Day,Hour,Group
0,4,2022-01-03,1001,1,296,5,09:21:14,419727,"GL H-Milch 3,5% 1L VLOG",1.0,2022-01-03 09:21:14,1,0,9,Milch
1,6,2022-01-03,1001,1,296,7,09:21:14,999999998025,Pfandartikel (Tandem) - EW,1.0,2022-01-03 09:21:14,1,0,9,Pfandartikel
2,19,2022-01-03,1001,1,300,1,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST
3,20,2022-01-03,1001,1,300,2,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST
4,75,2022-01-03,1001,1,305,3,09:29:46,999999998025,Pfandartikel (Tandem) - EW,1.0,2022-01-03 09:29:46,1,0,9,Pfandartikel


In [2]:
# frequency for each group
print(df.shape[0])
df.Group.value_counts()

3935939


Pfandartikel          2817389
Gurken ST              250097
Bananen Lose           243947
Milch                  235738
Broetchen              226873
BO-Laugenbreze 1ST     161895
Name: Group, dtype: int64

In [3]:
df = df.drop_duplicates(subset=['Date','StoreNumber', 'CashRegisterNumber', 'ReceiptNumber', 'Group'], keep='first')
display(df.head())
display(df.shape)
df.Group.value_counts()

,Unnamed: 0,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,ReceiptRowNumber,Time,ArticleNumber,ArticleName,Quantity,Timestamp,Month,Day,Hour,Group
0,4,2022-01-03,1001,1,296,5,09:21:14,419727,"GL H-Milch 3,5% 1L VLOG",1.0,2022-01-03 09:21:14,1,0,9,Milch
1,6,2022-01-03,1001,1,296,7,09:21:14,999999998025,Pfandartikel (Tandem) - EW,1.0,2022-01-03 09:21:14,1,0,9,Pfandartikel
2,19,2022-01-03,1001,1,300,1,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST
4,75,2022-01-03,1001,1,305,3,09:29:46,999999998025,Pfandartikel (Tandem) - EW,1.0,2022-01-03 09:29:46,1,0,9,Pfandartikel
5,78,2022-01-03,1001,1,306,1,09:32:07,419727,"GL H-Milch 3,5% 1L VLOG",1.0,2022-01-03 09:32:07,1,0,9,Milch


(2274858, 15)

Pfandartikel          1391044
Bananen Lose           239104
Gurken ST              215045
Broetchen              161431
Milch                  140315
BO-Laugenbreze 1ST     127919
Name: Group, dtype: int64

In [4]:
# drop articles with overwhelming quantity
df.drop(df[df.Group == "Pfandartikel"].index, inplace=True)
df.head()

,Unnamed: 0,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,ReceiptRowNumber,Time,ArticleNumber,ArticleName,Quantity,Timestamp,Month,Day,Hour,Group
0,4,2022-01-03,1001,1,296,5,09:21:14,419727,"GL H-Milch 3,5% 1L VLOG",1.0,2022-01-03 09:21:14,1,0,9,Milch
2,19,2022-01-03,1001,1,300,1,09:25:34,421791,BO-Laugenbreze 1ST,1.0,2022-01-03 09:25:34,1,0,9,BO-Laugenbreze 1ST
5,78,2022-01-03,1001,1,306,1,09:32:07,419727,"GL H-Milch 3,5% 1L VLOG",1.0,2022-01-03 09:32:07,1,0,9,Milch
6,92,2022-01-03,1001,1,307,1,09:35:19,419726,"GL H-Milch1,5%1L VLOG",10.0,2022-01-03 09:35:19,1,0,9,Milch
8,104,2022-01-03,1001,2,2,3,16:04:30,419726,"GL H-Milch1,5%1L VLOG",1.0,2022-01-03 16:04:30,1,0,16,Milch


# General, without separation

In [5]:
# integrate the identical transactions
integrated = df.groupby(['Date','StoreNumber', 'CashRegisterNumber', 'ReceiptNumber']).agg(Articles=('Group', lambda x:';'.join(x.unique()))).reset_index()
display(integrated.head())

# spilt the integrated transactions into several columns
split_1 = integrated['Articles'].str.split(';', expand=True).dropna(how='all', axis=1)
display(split_1.head())

for i in range(split_1.shape[1]):
    mini_article = split_1[split_1.iloc[:,i].notnull()]
    print(f"Transactions contain at least {i+1} article and the frequency is {mini_article.shape[0]}")

# filter transactions with at least 3 articles
split_1 = split_1[split_1.iloc[:,2].notnull()]
display(split_1.head())

# convert DataFrame to List
# delete the NoneType
res = []
def filter_list(_list):
    for item in _list:
        res.append(list(filter(None, item)))
filter_list(split_1.values.tolist())

transactions_from_df = [tuple(row) for row in res]
transactions_from_df[0:3]

# apriori algorithm
from efficient_apriori import apriori
itemsets, rules = apriori(transactions_from_df, min_support=0.25, min_confidence=0.5)
print("itemsets:",itemsets)
print("rules", rules)

,Date,StoreNumber,CashRegisterNumber,ReceiptNumber,Articles
0,2022-01-03,1001,1,296,Milch
1,2022-01-03,1001,1,300,BO-Laugenbreze 1ST
2,2022-01-03,1001,1,306,Milch
3,2022-01-03,1001,1,307,Milch
4,2022-01-03,1001,2,2,Milch


,0,1,2,3,4
0,Milch,None,None,None,None
1,BO-Laugenbreze 1ST,None,None,None,None
2,Milch,None,None,None,None
3,Milch,None,None,None,None
4,Milch,None,None,None,None


Transactions contain at least 1 article and the frequency is 750230
Transactions contain at least 2 article and the frequency is 118396
Transactions contain at least 3 article and the frequency is 14015
Transactions contain at least 4 article and the frequency is 1114
Transactions contain at least 5 article and the frequency is 59


,0,1,2,3,4
14,Milch,Gurken ST,Bananen Lose,None,None
34,Milch,BO-Laugenbreze 1ST,Broetchen,None,None
235,Gurken ST,Bananen Lose,Broetchen,None,None
278,Bananen Lose,Broetchen,BO-Laugenbreze 1ST,None,None
311,Milch,Gurken ST,Bananen Lose,None,None


itemsets: {1: {('Milch',): 8353, ('Gurken ST',): 10382, ('Bananen Lose',): 10685, ('BO-Laugenbreze 1ST',): 6026, ('Broetchen',): 7772}, 2: {('BO-Laugenbreze 1ST', 'Bananen Lose'): 3592, ('BO-Laugenbreze 1ST', 'Broetchen'): 3691, ('Bananen Lose', 'Broetchen'): 4950, ('Bananen Lose', 'Gurken ST'): 7698, ('Bananen Lose', 'Milch'): 6169, ('Broetchen', 'Gurken ST'): 4685, ('Gurken ST', 'Milch'): 6003}, 3: {('Bananen Lose', 'Gurken ST', 'Milch'): 4465}}
rules [{BO-Laugenbreze 1ST} -> {Bananen Lose}, {BO-Laugenbreze 1ST} -> {Broetchen}, {Broetchen} -> {Bananen Lose}, {Gurken ST} -> {Bananen Lose}, {Bananen Lose} -> {Gurken ST}, {Milch} -> {Bananen Lose}, {Bananen Lose} -> {Milch}, {Broetchen} -> {Gurken ST}, {Milch} -> {Gurken ST}, {Gurken ST} -> {Milch}, {Gurken ST, Milch} -> {Bananen Lose}, {Bananen Lose, Milch} -> {Gurken ST}, {Bananen Lose, Gurken ST} -> {Milch}, {Milch} -> {Bananen Lose, Gurken ST}]
